# Script de recalage d'image
Script pour recaler les images, paramètre ( chemin des images, les matrices des caméras, les caméras)
Renvoie une image recalé avec toutes les images combiné
## Première phase
Récupération des images à traiter
Variable global avec le nom des caméras

In [10]:
import cv2
import numpy as np
import os
import SimpleITK as sitk
import sys
import glob
from joblib import Parallel, delayed
from tqdm.notebook import tqdm,trange
from multiprocessing.pool import ThreadPool

Cameras = ("Camera1","Camera2","Camera3","Camera4")
setImages = "SET1"
CHEMINSET = os.path.join("..","DataSet","DAMAV_Jeux_Image",setImages)
cheminimage = os.path.join(CHEMINSET,"*.bmp")
CHEMINIMAGES = sorted(glob.glob(cheminimage))
print(CHEMINIMAGES)

['../DataSet/DAMAV_Jeux_Image/SET1/Image-000003_Camera1_2019-07-30_08-48-15-832541.bmp', '../DataSet/DAMAV_Jeux_Image/SET1/Image-000003_Camera2_2019-07-30_08-48-14-239599.bmp', '../DataSet/DAMAV_Jeux_Image/SET1/Image-000003_Camera3_2019-07-30_08-48-11-852455.bmp', '../DataSet/DAMAV_Jeux_Image/SET1/Image-000003_Camera4_2019-07-30_08-48-14-493286.bmp']


## Phase deux
Chargement des matrices des caméras
Renvoie un dictionnaire avec comme clé le nom de la caméra

In [11]:
cheminParam =[]
regex = os.path.join("Calibration","**","*.npz")
cheminParam = sorted(glob.glob(regex,recursive=True))

paramCam = dict()
for cam in Cameras:
    for fichierParam in cheminParam:
        if cam in fichierParam:
            with np.load(fichierParam) as data:
                paramCam[cam] = {"mtx" : data['mtx'],"dist" : data['dist']}
print(paramCam)

{'Camera1': {'mtx': array([[2.82568740e+03, 0.00000000e+00, 1.26325100e+03],
       [0.00000000e+00, 2.82586067e+03, 9.79291096e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]), 'dist': array([[-0.21907795,  0.25332612,  0.00183203, -0.00175609, -0.40434549]])}, 'Camera2': {'mtx': array([[2.83125446e+03, 0.00000000e+00, 1.24208969e+03],
       [0.00000000e+00, 2.82734145e+03, 1.06670254e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]), 'dist': array([[-2.16438861e-01,  3.86212514e-01, -4.21971083e-04,
        -2.88079811e-03, -8.87713469e-01]])}, 'Camera3': {'mtx': array([[2.81720047e+03, 0.00000000e+00, 1.21782943e+03],
       [0.00000000e+00, 2.81664248e+03, 1.00075666e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]), 'dist': array([[-0.18495925, -0.08103933,  0.00156822, -0.00254917,  0.64169507]])}, 'Camera4': {'mtx': array([[2.82158355e+03, 0.00000000e+00, 1.29136666e+03],
       [0.00000000e+00, 2.82303501e+03, 1.06018449e+03],
       

# Phase trois
Correction des images pour le recalage
Renvoie la zone d'interet (image)

In [17]:
def undistordImage(image:str,Cameras:tuple,paramCam:dict):

    for cam in Cameras:
        if cam in image:
            mtx = paramCam[cam]["mtx"]
            dist = paramCam[cam]["dist"]
            img = cv2.imread(image)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            h,  w = img.shape[:2]
            
            newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1.0, (w,h))
            # undistort
            undistorded = cv2.undistort(img, mtx, dist, None, newcameramtx)
            # crop the image
            x, y, w, h = roi
            roiundistorded = undistorded[y:y+h, x:x+w]
            roiundistorded = cv2.resize(roiundistorded,(2567, 1893),cv2.INTER_CUBIC)
            #cv2.imwrite("undistord"+cam+".bmp",undistorded)
            return roiundistorded

undistordedImg = Parallel(n_jobs=4,prefer="threads")(delayed(undistordImage)(CHEMINIMAGES[i],Cameras,paramCam)for i in tqdm(range(len(CHEMINIMAGES))))

  0%|          | 0/4 [00:00<?, ?it/s]

## Phase quatre
Definition de la metrique pour le recalage et de la fonction de recalage
Elle focntionne en parallèle avec joblib
Renvoie une image recalée

In [13]:
def computeMetric(fixed,moving):
    # First, set up "phony" registration

    R = sitk.ImageRegistrationMethod()
    R.SetOptimizerAsGradientDescentLineSearch(learningRate=1.0,numberOfIterations=1,convergenceMinimumValue=1e-5,convergenceWindowSize=5)
    R.SetInitialTransform(sitk.Transform(2,sitk.sitkIdentity)) # Transformation deliberately not using any initializer
    R.SetInterpolator(sitk.sitkLinear)

    #second, choose metric
    R.SetMetricAsMeanSquares()

    #third, get the metric value
    print(R.MetricEvaluate(fixed, moving))


In [14]:

def RecalageOpenCV(imageRef: any ,imageToAligned: any ,saveName:str):
    # Convert to grayscale.
    
    height, width = imageRef.shape
    
    # Create ORB detector with 5000 features.
    orb_detector = cv2.ORB_create(5000)

    # Find keypoints and descriptors.
    # The first arg is the image, second arg is the mask
    #  (which is not required in this case).
    
    kp1, d1 = orb_detector.detectAndCompute(imageToAligned, None)
    kp2, d2 = orb_detector.detectAndCompute(imageRef, None)

    
    # Match features between the two images.
    # We create a Brute Force matcher with
    # Hamming distance as measurement mode.
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
    
    # Match the two sets of descriptors.
    matches = matcher.match(d1, d2)
    
    # Sort matches on the basis of their Hamming distance.
    matches = sorted (matches,key = lambda x: x.distance)
    
    # Take the top 90 % matches forward.
    matches = matches[:int(len(matches)*0.9)]
    no_of_matches = len(matches)
    
    # Define empty matrices of shape no_of_matches * 2.
    p1 = np.zeros((no_of_matches, 2))
    p2 = np.zeros((no_of_matches, 2))
    
    for i in range(len(matches)):
        p1[i, :] = kp1[matches[i].queryIdx].pt
        p2[i, :] = kp2[matches[i].trainIdx].pt
    

    # Find the homography matrix.
    homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC)
    # Use this matrix to transform the
    # colored image wrt the reference image.
    transformed_img = cv2.warpPerspective(imageToAligned,
                        homography, (width, height))

    computeMetric(sitk.GetImageFromArray(np.float32(imageToAligned))  ,sitk.GetImageFromArray(np.float32(transformed_img)))

    # Save the output.
    #cv2.imwrite(saveName, transformed_img)
    return transformed_img


In [15]:
output = ("outV10.jpg","outV20.jpg","outV30.jpg")

homography = Parallel(n_jobs=4,prefer="threads")(delayed(RecalageOpenCV)(undistordedImg[0],undistordedImg[i],output[i-1])for i in range(1,4))

1946.2581406782126
396.94857378515684
2382.000613253141


## Phase cinq
Prend les images et les traites pour avoir une image recalé finale

In [16]:
images = []
for imageRecale in homography:
    images.append(imageRecale)
images.append(undistordedImg[0])

zeros = np.zeros((images[0].shape),dtype= "uint8" )

mask = []
for imageRecale in images:
    mask.append(cv2.compare(imageRecale,zeros,cv2.CMP_EQ))

fusion = zeros
for i in range(0,len(mask)):
    fusion = cv2.bitwise_or(mask[i],fusion)
masks = cv2.bitwise_not(fusion)

imagemasked = []
for imgeRecale in images:
    imagemasked.append(cv2.bitwise_and(imgeRecale,masks))

#cv2.imwrite("mask.jpg",masks)
end = cv2.merge(imagemasked)
cv2.imwrite(setImages+"outcolor.jpg",end)

True